In [4]:
# --- 1. 환경 설정 셀 (수정된 버전) ---
import os
import sys

# 작업 디렉토리를 올바른 위치로 변경
workspace_path = '/workspace'
os.chdir(workspace_path)

# RFdiffusion 경로를 파이썬이 인식하도록 추가
rfdiffusion_path = os.path.join(workspace_path, 'RFdiffusion')
if rfdiffusion_path not in sys.path:
    sys.path.append(rfdiffusion_path)
import time
import signal
import sys
import random
import string
import re
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

# Colab이 아닌 로컬 환경이므로, RFdiffusion 경로를 직접 추가
if 'RFdiffusion' not in sys.path:
    home_dir = os.path.expanduser("~")
    rfdiffusion_path = os.path.join(home_dir, 'RFdiffusion')
    sys.path.append(rfdiffusion_path)

os.environ["DGLBACKEND"] = "pytorch"

from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca
from colabdesign.rf.utils import fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

# --- Colab의 files.upload()를 대체하는 로컬 파일 처리 함수 ---
def get_pdb(pdb_code=None, use_upload=False):
    if use_upload:
        upload_widget = widgets.FileUpload(
            accept='.pdb', description='PDB 파일 업로드', button_style='info'
        )
        display(upload_widget)
        
        def wait_for_upload(widget):
            while len(widget.value) == 0: time.sleep(0.1)
            uploaded_filename = list(widget.value.keys())[0]
            pdb_content = widget.value[uploaded_filename]['content']
            pdb_filename = "tmp.pdb"
            with open(pdb_filename, "wb") as out: out.write(pdb_content)
            print(f"'{uploaded_filename}'이(가) 'tmp.pdb'로 저장되었습니다.")
            return pdb_filename
            
        return wait_for_upload(upload_widget)
    elif pdb_code is None or pdb_code == "":
        print("PDB 코드를 입력하거나 use_upload=True로 설정하세요.")
        return None
    elif os.path.isfile(pdb_code): return pdb_code
    elif len(pdb_code) == 4:
        pdb_filename = f"{pdb_code}.pdb1"
        if not os.path.isfile(pdb_filename):
            os.system(f"wget -qnc https://files.rcsb.org/download/{pdb_code}.pdb1.gz && gunzip -f {pdb_code}.pdb1.gz")
        return pdb_filename
    else:
        pdb_filename = f"AF-{pdb_code}-F1-model_v4.pdb"
        if not os.path.isfile(pdb_filename):
            os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/{pdb_filename}")
        return pdb_filename

def run_ananas(pdb_str, path, sym=None):
    pdb_filename = f"outputs/{path}/ananas_input.pdb"
    out_filename = f"outputs/{path}/ananas.json"
    os.makedirs(f"outputs/{path}", exist_ok=True)
    with open(pdb_filename,"w") as handle: handle.write(pdb_str)
    cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
    if sym is None: os.system(cmd)
    else: os.system(f"{cmd} {sym}")
    try:
        with open(out_filename,"r") as f: out = json.load(f)
        results, AU = out[0], out[-1]["AU"]
        group, chains, rmsd = AU["group"], AU["chain names"], results["Average_RMSD"]
        print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")
        C = np.array(results['transforms'][0]['CENTER'])
        A = [np.array(t["AXIS"]) for t in results['transforms']]
        new_lines = []
        for line in pdb_str.split("\n"):
            if line.startswith("ATOM"):
                chain = line[21:22]
                if chain in chains:
                    x = np.array([float(line[i:(i+8)]) for i in [30,38,46]])
                    if group[0] == "c": x = sym_it(x,C,A[0])
                    if group[0] == "d": x = sym_it(x,C,A[1],A[0])
                    coord_str = "".join([f"{a:8.3f}" for a in x])
                    new_lines.append(line[:30]+coord_str+line[54:])
            else: new_lines.append(line)
        return results, "\n".join(new_lines)
    except Exception as e:
        print(f"AnAnaS 결과 처리 중 오류 발생: {e}")
        return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):
    def run_command_and_get_pid(command):
        pid_file = '/dev/shm/pid'
        os.system(f'nohup {command} > /dev/null & echo $! > {pid_file}')
        with open(pid_file, 'r') as f: pid = int(f.read().strip())
        os.remove(pid_file)
        return pid
    def is_process_running(pid):
        try: os.kill(pid, 0)
        except OSError: return False
        else: return True
    run_output = widgets.Output()
    progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
    display(widgets.VBox([progress, run_output]))
    for n in range(steps):
        if os.path.isfile(f"/dev/shm/{n}.pdb"): os.remove(f"/dev/shm/{n}.pdb")
    pid = run_command_and_get_pid(command)
    try:
        fail = False
        for _ in range(num_designs):
            for n in range(steps):
                wait = True
                while wait and not fail:
                    time.sleep(0.1)
                    if os.path.isfile(f"/dev/shm/{n}.pdb"):
                        with open(f"/dev/shm/{n}.pdb", "r") as f: pdb_str = f.read()
                        if "TER" in pdb_str or "ENDMDL" in pdb_str: wait = False
                        elif not is_process_running(pid): fail = True
                    elif not is_process_running(pid): fail = True
                if fail:
                    progress.bar_style = 'danger'; progress.description = "failed"
                    break
                else:
                    progress.value = (n+1) / steps
                    if visual != "none":
                        with run_output:
                            run_output.clear_output(wait=True)
                            if visual == "image":
                                xyz, bfact = get_ca(f"/dev/shm/{n}.pdb", get_bfact=True)
                                fig = plt.figure(); fig.set_dpi(100);fig.set_figwidth(6);fig.set_figheight(6)
                                ax1 = fig.add_subplot(111);ax1.set_xticks([]);ax1.set_yticks([])
                                plot_pseudo_3D(xyz, c=bfact, cmin=0.5, cmax=0.9, ax=ax1); plt.show()
                            if visual == "interactive":
                                view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
                                view.addModel(pdb_str,'pdb'); view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
                                view.zoomTo(); view.show()
                if os.path.exists(f"/dev/shm/{n}.pdb"): os.remove(f"/dev/shm/{n}.pdb")
            if fail:
                progress.bar_style = 'danger'; progress.description = "failed"
                break
        while is_process_running(pid): time.sleep(0.1)
    except KeyboardInterrupt:
        os.kill(pid, signal.SIGTERM); progress.bar_style = 'danger'; progress.description = "stopped"
    
def run_diffusion(contigs, path, pdb=None, iterations=50, symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, num_designs=1, visual="none"):
    full_path = f"outputs/{path}"; os.makedirs(full_path, exist_ok=True)
    opts = [f"inference.output_prefix={full_path}", f"inference.num_designs={num_designs}"]
    if chains == "": chains = None
    if symmetry in ["auto","cyclic","dihedral"]:
        if symmetry == "auto": sym, copies = None, 1
        else: sym, copies = {"cyclic":(f"c{order}",order), "dihedral":(f"d{order}",order*2)}[symmetry]
    else: symmetry = None; sym, copies = None, 1
    contigs_list = contigs.replace(","," ").replace(":"," ").split()
    is_fixed, is_free = False, False; fixed_chains = []
    for contig in contigs_list:
        for x in contig.split("/"):
            a = x.split("-")[0]
            if a and a[0].isalpha():
                is_fixed = True
                if a[0] not in fixed_chains: fixed_chains.append(a[0])
            if a.isnumeric(): is_free = True
    if len(contigs_list) == 0 or not is_free: mode = "partial"
    elif is_fixed: mode = "fixed"
    else: mode = "free"
    if mode in ["partial","fixed"]:
        pdb_str = pdb_to_string(pdb, chains=chains)
        if symmetry == "auto":
            a, pdb_str = run_ananas(pdb_str, path)
            if a is None: print(f'ERROR: no symmetry detected'); symmetry = None; sym, copies = None, 1
            else:
                if a["group"][0] == "c": symmetry = "cyclic"; sym, copies = a["group"], int(a["group"][1:])
                elif a["group"][0] == "d": symmetry = "dihedral"; sym, copies = a["group"], 2 * int(a["group"][1:])
                else: print(f'ERROR: detected symm ({a["group"]}) not supported'); symmetry = None; sym, copies = None, 1
        elif mode == "fixed": pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)
        pdb_filename = f"{full_path}/input.pdb"
        with open(pdb_filename, "w") as handle: handle.write(pdb_str)
        parsed_pdb = parse_pdb(pdb_filename)
        opts.append(f"inference.input_pdb={pdb_filename}")
        if mode in ["partial"]:
            iterations = int(80 * (iterations / 200)); opts.append(f"diffuser.partial_T={iterations}")
            contigs_list = fix_partial_contigs(contigs_list, parsed_pdb)
        else: opts.append(f"diffuser.T={iterations}"); contigs_list = fix_contigs(contigs_list, parsed_pdb)
    else:
        opts.append(f"diffuser.T={iterations}"); parsed_pdb = None
        contigs_list = fix_contigs(contigs_list, parsed_pdb)
    if hotspot is not None and hotspot != "": opts.append(f"ppi.hotspot_res=[{hotspot}]")
    if sym is not None:
        sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
        if add_potential: sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'", "potentials.olig_intra_all=True","potentials.olig_inter_all=True", "potentials.guide_scale=2","potentials.guide_decay=quadratic"]
        opts = sym_opts + opts; contigs_list = sum([contigs_list] * copies,[])
    opts.append(f"'contigmap.contigs=[{' '.join(contigs_list)}]'")
    opts += ["inference.dump_pdb=True","inference.dump_pdb_path='/dev/shm'"]
    print("mode:", mode); print("output:", full_path); print("contigs:", contigs_list)
    run_script_path = "/workspace/RFdiffusion/run_inference.py"
    opts_str = " ".join(opts)
    cmd = f"python {run_script_path} {opts_str}"
    print(cmd)
    run(cmd, iterations, num_designs, visual=visual)
    for n in range(num_designs):
        pdbs = [f"outputs/traj/{path}_{n}_pX0_traj.pdb", f"outputs/traj/{path}_{n}_Xt-1_traj.pdb", f"{full_path}_{n}.pdb"]
        for pdb_file in pdbs:
            if os.path.exists(pdb_file):
                with open(pdb_file,"r") as handle: pdb_str = handle.read()
                with open(pdb_file,"w") as handle: handle.write(fix_pdb(pdb_str, contigs_list))
    return contigs_list, copies

print("✅ Setup cell is ready.")

✅ Setup cell is ready.


In [ ]:
# --- 필요한 라이브러리 import ---
import os
import random
import string

# --- 바인더 디자인 실행 셀 (1단계: 백본 생성) ---

# 1. 파라미터 설정
# -----------------------------------------------------------------
name = "0821-binder-backbones"
contigs = "C311-391/0 100-100"
hotspot = "C331,C360,C378,C381,C323"
num_designs = 50  # 예시로 개수를 줄임 (필요시 200으로 다시 조절)
pdb_filename = "4k9e_c.pdb"

# --- 중간 저장 파라미터 ---
total_iterations = 500
save_interval = 50
# -----------------------------------------------------------------


# 2. PDB 파일 경로 및 기본 출력 폴더 설정
pdb = os.path.join("/workspace", pdb_filename)
base_name = name
output_dir = f"outputs/{base_name}"

# 중복 방지 폴더명 생성
while os.path.exists(output_dir):
    random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
    base_name = f"{name}_{random_suffix}"
    output_dir = f"outputs/{base_name}"

print(f"✅ [단계 1] 백본이 저장될 기본 폴더: {output_dir}")


# 3. RFdiffusion 반복 실행
# -----------------------------------------------------------------
print(f"🚀 RFdiffusion으로 백본 생성을 시작합니다 (간격: {save_interval}, 최종: {total_iterations}).")

for current_iter in range(save_interval, total_iterations + 1, save_interval):
    print(f"\n{'='*50}\n▶️ 현재 Iteration 단계 실행: {current_iter}")
    
    iter_folder_name = f"iter_{current_iter}"
    iter_file_prefix = f"design_iter_{current_iter}"
    path = os.path.join(base_name, iter_folder_name, iter_file_prefix)

    print(f"   - 저장 경로: outputs/{path}_0.pdb ...")

    flags = {
        "contigs": contigs, "pdb": pdb, "iterations": int(current_iter),
        "hotspot": hotspot, "path": path, "num_designs": int(num_designs),
        "visual": "none", "symmetry": "none", "order": 1, "chains": "",
        "add_potential": True
    }

    for k, v in flags.items():
        if isinstance(v, str):
            flags[k] = v.replace("'", "").replace('"', '')
    
    run_diffusion(**flags) # 이 함수가 실행된다고 가정

print(f"\n{'='*50}")
print(f"✅ [단계 1] 백본 생성 완료! 결과는 '{output_dir}' 폴더에 저장되었습니다.")

✅ [단계 1] 백본이 저장될 기본 폴더: outputs/0821-binder-backbones
🚀 RFdiffusion으로 백본 생성을 시작합니다 (간격: 50, 최종: 500).

▶️ 현재 Iteration 단계 실행: 50
   - 저장 경로: outputs/0821-binder-backbones/iter_50/design_iter_50_0.pdb ...
mode: fixed
output: outputs/0821-binder-backbones/iter_50/design_iter_50
contigs: ['C311-391', '100-100']
python /workspace/RFdiffusion/run_inference.py inference.output_prefix=outputs/0821-binder-backbones/iter_50/design_iter_50 inference.num_designs=50 inference.input_pdb=outputs/0821-binder-backbones/iter_50/design_iter_50/input.pdb diffuser.T=50 ppi.hotspot_res=[C331,C360,C378,C381,C323] 'contigmap.contigs=[C311-391 100-100]' inference.dump_pdb=True inference.dump_pdb_path='/dev/shm'



▶️ 현재 Iteration 단계 실행: 100
   - 저장 경로: outputs/0821-binder-backbones/iter_100/design_iter_100_0.pdb ...
mode: fixed
output: outputs/0821-binder-backbones/iter_100/design_iter_100
contigs: ['C311-391', '100-100']
python /workspace/RFdiffusion/run_inference.py inference.output_prefix=outputs/0821-binder-backbones/iter_100/design_iter_100 inference.num_designs=50 inference.input_pdb=outputs/0821-binder-backbones/iter_100/design_iter_100/input.pdb diffuser.T=100 ppi.hotspot_res=[C331,C360,C378,C381,C323] 'contigmap.contigs=[C311-391 100-100]' inference.dump_pdb=True inference.dump_pdb_path='/dev/shm'


In [ ]:
# --- 필요한 라이브러리 import ---
import os
import subprocess

# --- 디자인 및 예측 실행 셀 (2단계: 서열 디자인 및 검증) ---

# 1. 파라미터 설정
# -----------------------------------------------------------------
# 단계 1에서 결과가 저장된 기본 폴더 경로를 정확하게 입력합니다.
# 위 코드에서 `output_dir`로 출력된 최종 폴더명을 사용하세요.
rfdiffusion_results_dir = "outputs/0821-binder-backbones" # ⚠️ 여기를 확인하고 수정하세요!

# ProteinMPNN과 ColabFold(ColabDesign) 실행 스크립트 또는 명령어 경로
# 실제 환경에 맞게 수정해야 합니다.
protein_mpnn_script = "/path/to/ProteinMPNN/protein_mpnn_run.py"
colabfold_command = "colabfold_batch" # colabfold가 환경에 설치되어 있다고 가정
# -----------------------------------------------------------------

print(f"✅ [단계 2] 서열 디자인 및 구조 예측을 시작합니다.")
print(f"   - 대상 폴더: {rfdiffusion_results_dir}")

# 2. 모든 PDB 파일을 순회하며 작업 실행
if not os.path.isdir(rfdiffusion_results_dir):
    print(f"⚠️ 에러: '{rfdiffusion_results_dir}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요.")
else:
    # os.walk를 사용하여 모든 하위 폴더를 탐색
    for root, dirs, files in os.walk(rfdiffusion_results_dir):
        for file in files:
            if file.endswith(".pdb"):
                pdb_path = os.path.join(root, file)
                
                print(f"\n{'--'*25}\n▶️ 처리 중인 파일: {pdb_path}")

                # --- 🧬 ProteinMPNN 실행 ---
                # 결과가 저장될 경로 설정 (PDB 파일과 같은 폴더에 저장)
                mpnn_output_path = os.path.join(root, "mpnn_outputs")
                os.makedirs(mpnn_output_path, exist_ok=True)
                
                mpnn_command = [
                    "python", protein_mpnn_script,
                    "--pdb_path", pdb_path,
                    "--out_folder", mpnn_output_path,
                    "--num_seq_per_target", "1", # 백본 당 1개의 서열 생성 (필요시 조절)
                    "--sampling_temp", "0.1"
                ]
                
                print(f"   - (1/2) ProteinMPNN으로 서열 디자인 실행...")
                # subprocess.run(mpnn_command, capture_output=True, text=True) # 실제 실행 시 주석 해제

                # --- 🤖 ColabDesign (ColabFold) 실행 ---
                # ProteinMPNN 결과(FASTA 파일)를 입력으로 사용
                # 실제 파일 이름은 ProteinMPNN 출력 형식에 맞게 수정 필요
                base_name = os.path.splitext(file)[0]
                fasta_path = os.path.join(mpnn_output_path, "seqs", f"{base_name}.fa")
                
                colabdesign_output_path = os.path.join(root, "colabdesign_outputs")
                # os.makedirs(colabdesign_output_path, exist_ok=True) # colabfold_batch가 자동으로 폴더 생성

                # if os.path.exists(fasta_path): # 실제 실행 시 주석 해제
                colabfold_run_command = [
                    colabfold_command,
                    fasta_path, # ProteinMPNN이 생성한 fasta 파일
                    colabdesign_output_path
                ]
                
                print(f"   - (2/2) ColabDesign으로 구조 예측 및 검증 실행...")
                # subprocess.run(colabfold_run_command, capture_output=True, text=True) # 실제 실행 시 주석 해제
                # else: # 실제 실행 시 주석 해제
                #    print(f"   - ⚠️ 경고: ProteinMPNN의 결과 파일({fasta_path})을 찾을 수 없어 ColabDesign을 건너뜁니다.")


print(f"\n{'='*50}")
print("✅ [단계 2] 모든 백본에 대한 서열 디자인 및 구조 예측 작업이 완료되었습니다.")
print("📊 이제 각 colabdesign_outputs 폴더에 있는 pLDDT, RMSD 등의 점수를 확인하여 최고의 디자인을 선별하세요.")

In [ ]:
# rmsd & i_pae 조정해서 디자인 선별

In [3]:
import pandas as pd

# 1. 분석할 파일 경로와 필터링 조건 설정
results_csv_path = "outputs/0819-50-200/0819-50-200/mpnn_results.csv"
RMSD_THRESHOLD = 3.0
IPAE_THRESHOLD = 15.0

print(f"'{results_csv_path}' 파일에서 결과 분석을 시작합니다.")
print(f"✅ 적용할 필터링 조건: rmsd < {RMSD_THRESHOLD} AND i_pae < {IPAE_THRESHOLD}")

# 2. CSV 파일 읽기
try:
    df_all = pd.read_csv(results_csv_path)
    
    # 3. 설정된 조건으로 데이터 필터링
    df_filtered = df_all[(df_all['rmsd'] < RMSD_THRESHOLD) & (df_all['i_pae'] < IPAE_THRESHOLD)].copy()
    
    print(f"\n📊 총 {len(df_filtered)}개의 디자인이 설정된 조건을 만족했습니다.")

    # 4. 필터링된 결과가 있으면 요약 및 목록 출력
    if not df_filtered.empty:
        backbone_counts = df_filtered['design'].value_counts()
        
        # ⭐️⭐️⭐️ 요청하신 추가 부분 ⭐️⭐️⭐️
        print(f"🔬 총 {len(backbone_counts)}개의 Backbone에서 디자인이 선별되었습니다.")
        
        # Backbone별 통과된 디자인 개수 요약 (내림차순)
        print("\nBackbone별 통과된 디자인 개수 (많은 순):")
        for design_id, count in backbone_counts.items():
            print(f"  - Backbone 'design{design_id}': {count}개 통과")
        
        # RMSD 기준으로 오름차순 정렬
        df_sorted = df_filtered.sort_values(by='rmsd', ascending=True)
        
        # 최종 선별된 디자인 목록 전체 출력
        print("\n📋 선별된 디자인 전체 목록 (RMSD 오름차순):")
        print(df_sorted[['Unnamed: 0', 'design', 'plddt', 'i_pae', 'rmsd']].to_string(index=False))
        
    else:
        print("\n조건을 만족하는 디자인이 없습니다.")

except FileNotFoundError:
    print(f"\n🚨 에러: 결과 파일({results_csv_path})을 찾을 수 없습니다. 파일 경로를 확인해주세요.")

'outputs/0819-50-200/0819-50-200/mpnn_results.csv' 파일에서 결과 분석을 시작합니다.
✅ 적용할 필터링 조건: rmsd < 3.0 AND i_pae < 15.0

📊 총 9개의 디자인이 설정된 조건을 만족했습니다.
🔬 총 4개의 Backbone에서 디자인이 선별되었습니다.

Backbone별 통과된 디자인 개수 (많은 순):
  - Backbone 'design33': 5개 통과
  - Backbone 'design147': 2개 통과
  - Backbone 'design106': 1개 통과
  - Backbone 'design130': 1개 통과

📋 선별된 디자인 전체 목록 (RMSD 오름차순):
 Unnamed: 0  design    plddt     i_pae     rmsd
        268      33 0.912716  8.088408 1.670351
        270      33 0.926660  7.647270 1.776981
        269      33 0.922526  8.518820 1.920088
        264      33 0.921417  8.165914 2.084753
       1176     147 0.916483 10.029659 2.174413
        265      33 0.924587  8.333292 2.207138
        849     106 0.932891 12.802895 2.514534
       1183     147 0.809654 13.867611 2.551431
       1045     130 0.819364  9.935198 2.893448
